In [1]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

In [2]:
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///aid_data.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(db.engine, reflect=True)

aid = Base.classes.aid_data

C:\Users\llreb\Anaconda3\envs\PythonData\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
year = 1996

In [3]:
year = 1996
sel = [
        aid.country_name,
        aid.latitude,
        aid.longitude,
        aid.transaction_type_name,
        func.sum(aid.constant_amount),
        aid.fiscal_year        
    ]

results = db.session.query(*sel).filter(aid.fiscal_year == year).filter(aid.transaction_type_name == "Obligations").group_by(aid.country_name, aid.fiscal_year, aid.transaction_type_name)

# Create a dictionary entry for each row of information

map_data = []
for result in results:
    aid_dict = {}
    aid_dict["country"] = result[0]
    aid_dict["latitude"] = result[1]
    aid_dict["longitude"] = result[2]
    aid_dict["transaction type"] = result[3]
    aid_dict["amount"] = result[4]
    aid_dict["year"] = result[5]
    map_data.append(aid_dict)
print(map_data)

[{'country': 'Afghanistan', 'latitude': 33.93911, 'longitude': 67.709953, 'transaction type': 'Obligations', 'amount': 24036781.0, 'year': 1996.0}, {'country': 'Albania', 'latitude': 41.153332, 'longitude': 20.168331, 'transaction type': 'Obligations', 'amount': 49025874.0, 'year': 1996.0}, {'country': 'Algeria', 'latitude': 28.033886, 'longitude': 1.6596259999999998, 'transaction type': 'Obligations', 'amount': 214396.0, 'year': 1996.0}, {'country': 'Angola', 'latitude': -11.202691999999999, 'longitude': 17.873887, 'transaction type': 'Obligations', 'amount': 133784437.0, 'year': 1996.0}, {'country': 'Antigua and Barbuda', 'latitude': 17.060816, 'longitude': -61.796428000000006, 'transaction type': 'Obligations', 'amount': 258553.0, 'year': 1996.0}, {'country': 'Argentina', 'latitude': -38.416097, 'longitude': -63.616671999999994, 'transaction type': 'Obligations', 'amount': 24748734.0, 'year': 1996.0}, {'country': 'Armenia', 'latitude': 40.069099, 'longitude': 45.038189, 'transaction

In [4]:
stmt = db.session.query(aid).statement
df = pd.read_sql_query(stmt, db.session.bind)
df.head()

,index,country_name,latitude,longitude,region_name,income_group_name,implementing_agency_name,channel_category_name,dac_sector_name,assistance_category_name,transaction_type_name,fiscal_year,constant_amount,USG_sector_name
0,0,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,Department of Defense,Government,"Conflict, Peace, and Security",Military,Obligations,2011.0,1.073199e+10,Stabilization Operations and Security Sector R...
1,1,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,Department of Defense,Government,"Conflict, Peace, and Security",Military,Obligations,2012.0,9.799467e+09,Stabilization Operations and Security Sector R...
2,4,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,Department of Defense,Government,"Conflict, Peace, and Security",Military,Obligations,2013.0,7.223113e+09,Stabilization Operations and Security Sector R...
3,7,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,Department of Defense,Government,"Conflict, Peace, and Security",Military,Obligations,2010.0,7.124546e+09,Stabilization Operations and Security Sector R...
4,9,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,Department of Defense,Government,"Conflict, Peace, and Security",Military,Obligations,2015.0,6.104972e+09,Stabilization Operations and Security Sector R...


In [5]:
# Filter the data based on the year
year = 2016

aid_data = df.loc[(df["fiscal_year"] == year) & (df["transaction_type_name"] == "Disbursements"), ["country_name", "latitude", "longitude", "transaction_type_name", "constant_amount", "fiscal_year"]]
# Format the data to send as json
aid_data.head()



,country_name,latitude,longitude,transaction_type_name,constant_amount,fiscal_year


In [7]:
#read the list of countries
countries=pd.read_csv("./country_name.csv")
countries.head()

,Unnamed: 0,country,year_count
0,0,Afghanistan,19690
1,1,Albania,4330
2,2,Algeria,897
3,3,Angola,5676
4,4,Anguilla,5


In [4]:
year = 1996
sel = [
        aid.assistance_category_name,
        aid.transaction_type_name,
        db.func.sum(aid.constant_amount),
        aid.fiscal_year        
    ]
results = db.session.query(*sel).filter(aid.fiscal_year == year).filter(aid.transaction_type_name == "Obligations").group_by( aid.fiscal_year, aid.assistance_category_name)


In [5]:
bar_data = []
for result in results:
    bar_dict = {}
    bar_dict["assistance type"] = result[0]
    bar_dict["transaction type"] = result[1]
    bar_dict["amount"] = result[2]
    bar_dict["year"] = result[3]
    bar_data.append(bar_dict)
bar_data

[{'assistance type': 'Economic',
  'transaction type': 'Obligations',
  'amount': 8307155432.0,
  'year': 1996.0},
 {'assistance type': 'Military',
  'transaction type': 'Obligations',
  'amount': 6826344198.0,
  'year': 1996.0}]